<a href="https://colab.research.google.com/github/Chiraagkv/ASL/blob/main/experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASL to Text
An application to convert sign language to text.

## 1. Problem Defination

Everyone cannot understand sign language, to help everyone to read sign language, an AI based solution to convert sign language to text will be ideal solution.

## 2. Data

Using [ASL Alphabet dataset](https://www.kaggle.com/grassknoted/asl-alphabet) from [Kaggle.com](https://www.kaggle.com)

in the form of data we have Images (unstructured data).

The data structure is:

```bash
+- asl_alphabet_test 
| +- asl_alphabet_test
| | +- images.jpg
|
+- asl_alphabet_train
| +- asl_alphabet_train
| | +- <CLASSNAME>
| | | +- images.jpg
```

## 3. Evaluation

As our problem is multiclass classification, we will be using evaluation methods for multi-class classification

## 4. Features

* Unstructured Data
* 29 classes/different signs
* about 87K images in the train folder (total)
* 28 images for the test set

In [11]:
# imports
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

download the datset

In [ ]:
!pip install kaggle  

In [12]:
!mkdir /root/.kaggle/
!touch /root/.kaggle/kaggle.json
!echo '{"username":"yashpawarp","key":"bf231966c472597c9b7dfd0c64730dc9"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d grassknoted/asl-alphabet --unzip

mkdir: cannot create directory ‘/root/.kaggle/’: File exists
 99% 1.02G/1.03G [00:08<00:00, 139MB/s]
100% 1.03G/1.03G [00:09<00:00, 122MB/s]


Unzip the data

In [4]:
from zipfile import ZipFile

with ZipFile('/content/asl-alphabet.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

### Load the data

In [18]:
def unbatchify(batch):
    '''
    Takes a batched dataset of (image, label) Tensors and returns separate arrays of imags and labels
    '''
    images = []
    labels = []
    # Loop through batched data
    for image, label in batch.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

In [15]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/asl_alphabet_train/asl_alphabet_train',  # data directory
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=42,
    validation_split=None,
    subset=None,
    interpolation="bilinear"
)

Found 87000 files belonging to 29 classes.


In [16]:
from sklearn.model_selection import StratifiedShuffleSplit

def stratify_split(X, y, test_size):
    """
    Split the data using StratifiedShuffleSplit
    """
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=42)

    for train_index, test_index in split.split(X, y):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]

    return X_train, y_trian, X_test, y_test

In [ ]:
# unbatchify crashes the session